# **AIAuto L08: Deep learning a steering controller.**
v2021.01


---

Version history:  
v2021.01 - initial version

---

### **import dependencies**

In [ ]:
# Importing necessary libraries
import numpy as np                # for linear algebra
import tensorflow as tf           # for neural models
import time                       # for measuring time
import zipfile                    # for handling zip archives
import PIL                        # for image handling
from keras import backend as K
import os

In [ ]:
import matplotlib.pyplot as plt   # for plotting images

In [ ]:
import pandas as pd
import csv

### get data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!unzip TRAININGOLD3.zip
!unzip VALIDATIONOLD3.zip

In [ ]:
import cv2
image = cv2.imread('/content/drive/MyDrive/Train2/00104846.png')

In [ ]:
plt.imshow(image)

In [ ]:
# Definitions
TRIM_TOP_PX = 32
GRAYSCALE = True
IMG_SHAPE = (120-TRIM_TOP_PX, 320, 1 if GRAYSCALE else 3)

TRAIN_SET_PATH = '/content/drive/MyDrive/TRAININGOLD3'
DEV_SET_PATH = '/content/drive/MyDrive/VALIDATIONOLD3'

In [ ]:
def plot_images(images):
    assert isinstance(images, (list, tuple, np.ndarray))    
    cols = min(6, len(images))
    rows = 1 + (len(images)-1)//cols
    plt.figure(figsize=(3.2*cols, 1.2*rows))
    for n, image in enumerate(images):
        plt.subplot(rows, cols, n+1)
        plt.xticks([], [])
        plt.yticks([], [])
        # plt.xlabel(f'{pred_titles[n]}', size=14, c='blue')
        if image.shape[2] == 1:
            image = np.concatenate((image, image, image), axis=2)
        plt.imshow(image)
    plt.tight_layout()
    plt.show()

In [ ]:
FIRST_TRAIN_IMG = "00104846.png"

image = tf.keras.preprocessing.image.load_img(f"{TRAIN_SET_PATH}/{FIRST_TRAIN_IMG}",
                                              color_mode="grayscale" if GRAYSCALE else "rgb")
img_arr = tf.keras.preprocessing.image.img_to_array(image)/255.0
img_arr = img_arr[TRIM_TOP_PX:,:,:]

img_arr = np.expand_dims(img_arr, axis=0)
print(img_arr.shape)
plot_images(img_arr)

In [ ]:
labels_df = pd.read_csv(f"{TRAIN_SET_PATH}/control_data.csv")
labels_df.head()

In [ ]:
labels_df.std(axis=0)

frame         16231.589902
speed             0.023486
throttle          0.000000
steer             0.031193
brake             0.000000
hand_brake        0.000000
dtype: float64

In [ ]:
STEER_NORM_FACTOR = 0.05

In [ ]:
def generate_generator(file_path, skip_rows=1):
    def fetch_data_pair():
        with open(f'{file_path}/control_data.csv', 'r') as csvfile:
            data = csv.reader(csvfile, delimiter=',')
            for index, row in enumerate(data):
                if index < skip_rows:
                    continue
                image = tf.keras.preprocessing.image.load_img(f'{file_path}/{int(row[0]):08d}.png',
                                                              color_mode="grayscale" if GRAYSCALE else "rgb")
                img_arr = tf.keras.preprocessing.image.img_to_array(image)
                img_arr = img_arr[TRIM_TOP_PX:,:,:]
                yield img_arr/255.0, float(row[3])/STEER_NORM_FACTOR
    return fetch_data_pair

In [ ]:
train_set = tf.data.Dataset.from_generator(generate_generator(TRAIN_SET_PATH), output_types=(tf.float32, tf.float64),
                                           output_shapes=(tf.TensorShape(IMG_SHAPE), tf.TensorShape(())))
dev_set = tf.data.Dataset.from_generator(generate_generator(DEV_SET_PATH), output_types=(tf.float32, tf.float64),
                                         output_shapes=(tf.TensorShape(IMG_SHAPE), tf.TensorShape(())))

In [ ]:
for i, (feature, label) in enumerate(train_set.take(500)):
    if i%50==0:
        plot_images([feature])
        print(label.numpy())

In [ ]:
# Neural network model
def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(8, (5,5), activation='elu', input_shape=IMG_SHAPE),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(16, (3,3), activation='elu'),
        tf.keras.layers.MaxPooling2D((2,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation='elu'),
        tf.keras.layers.Conv2D(16, (3,3), activation='elu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(16, (3,3), activation='elu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(8, (3,3), activation='elu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='linear', name='LAYER_FINAL_fc')
    ])

    model.summary()
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(lr=0.001, decay=0.001),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
m01 = build_model()

In [ ]:
history = m01.fit(
    train_set.batch(96),
    validation_data=dev_set.batch(32),
    epochs=30,
    verbose=1,
    shuffle=True)

In [ ]:
for i, (feature, label) in enumerate(train_set.take(500)):
    if i%50==0:
        plot_images([feature])
        print(label.numpy())
        print(m01.predict(np.expand_dims(feature, axis=0))[0,0])

In [ ]:
m01.save('TRAININGOLD3.h5')

Now you can download the `m01.h5` model manually from the left hand file explorer bar.

In [ ]:
reconstructed_model = tf.keras.models.load_model("Train.h5")
for i, (feature, label) in enumerate(train_set.take(500)):
    if i%50==0:
        plot_images([feature])
        print(label.numpy())
        print(reconstructed_model.predict(np.expand_dims(feature, axis=0))[0,0])

https://www.tensorflow.org/install/source#gpu

https://towardsdatascience.com/installing-tensorflow-gpu-in-ubuntu-20-04-4ee3ca4cb75d

(https://gist.github.com/kmhofmann/cee7c0053da8cc09d62d74a6a4c1c5e4)